In [ ]:
import kotlinx.serialization.json.Json
import java.io.File

val json = Json.parseToJsonElement(
    File("dataset.json").readText()
)

In [ ]:
import io.github.lexadiky.jsonquery.query

// Using path
println(json.query { path("shelter.name") }) // "Purrfect Paws Cat Shelter"

// Using path with index
println(json.query { path("shelter.cats")[0].path("name") }) // "Mittens"

// Using path with in array lookup
println(json.query { path("shelter.cats.name") }) // ["Mittens","Shadow","Mumpkin"]

// Using path with filter
println(json.query { path("shelter.cats.name").filterT<String> { it.startsWith("M") } }) // ["Mittens","Mumpkin"]